<a href="https://colab.research.google.com/github/BRV12G/Final_year_Project/blob/main/svm_old_diet_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the uploaded dataset to inspect its structure
file_path = '/content/nutition_updated (1).csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset along with some summary information
data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Person ID                20000 non-null  object
 1   Gender                   20000 non-null  object
 2   Age                      20000 non-null  int64 
 3   Occupation               20000 non-null  object
 4   Sleep Duration           20000 non-null  int64 
 5   Quality of Sleep         20000 non-null  object
 6   Physical Activity Level  20000 non-null  object
 7   Stress Level             20000 non-null  object
 8   BMI Category             20000 non-null  object
 9   Blood Pressure Category  20000 non-null  object
 10  Systolic                 20000 non-null  int64 
 11  Diastolic                20000 non-null  int64 
 12  Heart Rate               20000 non-null  int64 
 13  Daily Steps              20000 non-null  int64 
 14  Sleep Disorder           14951 non-nul

(  Person ID  Gender  Age Occupation  Sleep Duration Quality of Sleep  \
 0        P1  Female   50    Teacher               5        Excellent   
 1        P2  Female   31  Scientist              10             Good   
 2        P3    Male   36  Scientist              10        Excellent   
 3        P4    Male   44    Manager               9             Fair   
 4        P5    Male   64     Artist               6             Fair   
 
   Physical Activity Level Stress Level BMI Category Blood Pressure Category  \
 0                  Medium       Medium   Overweight            Hypertension   
 1                    High         High  Underweight         Prehypertension   
 2                     Low          Low       Normal                  Normal   
 3                     Low         High   Overweight                  Normal   
 4                     Low         High   Overweight            Hypertension   
 
    Systolic  Diastolic  Heart Rate  Daily Steps Sleep Disorder Health Status 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Drop irrelevant columns (e.g., 'Person ID') if they do not contribute to prediction
data = data.drop(columns=['Person ID', 'Nutritional Needs'])

# Encode categorical features
label_encoders = {}
for column in data.select_dtypes(include='object').columns:
    if column != 'Health Status':  # Target variable, to be encoded separately
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))
        label_encoders[column] = le

# Handle missing values for "Sleep Disorder" using SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
data['Sleep Disorder'] = imputer.fit_transform(data[['Sleep Disorder']])

# Encode the target label 'Health Status'
target_encoder = LabelEncoder()
data['Health Status'] = target_encoder.fit_transform(data['Health Status'])

# Split the data into features (X) and target (y)
X = data.drop(columns=['Health Status'])
y = data['Health Status']

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((16000, 14), (4000, 14), (16000,), (4000,))

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Initialize the SVM classifier with a linear kernel
svm_model = SVC(kernel='linear', random_state=42)

# Train the model
svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=target_encoder.classes_)

print(accuracy, classification_rep)


0.9775               precision    recall  f1-score   support

     Healthy       0.93      0.95      0.94       750
   Unhealthy       0.99      0.98      0.99      3250

    accuracy                           0.98      4000
   macro avg       0.96      0.97      0.96      4000
weighted avg       0.98      0.98      0.98      4000



In [ ]:
import numpy as np

def predict_health_status(input_data):
    """
    Predict the health status of a user based on their input data.

    Parameters:
    input_data (dict): Dictionary containing feature values similar to original dataset structure.

    Returns:
    str: Predicted health status ('Healthy' or 'Unhealthy').
    """
    # Create a DataFrame from the input data
    input_df = pd.DataFrame([input_data])

    # Encode categorical features using the previously fitted LabelEncoders
    for column, le in label_encoders.items():
        input_df[column] = le.transform(input_df[column].astype(str))

    # Handle missing values
    input_df['Sleep Disorder'] = imputer.transform(input_df[['Sleep Disorder']])

    # Scale features using the previously fitted StandardScaler
    input_scaled = scaler.transform(input_df)

    # Predict health status using the trained SVM model
    prediction = svm_model.predict(input_scaled)

    # Decode the prediction to its original label
    predicted_status = target_encoder.inverse_transform(prediction)[0]

    return predicted_status

# Sample input for testing the function (based on the dataset structure)
sample_input = {
    'Gender': 'Female',
    'Age': 35,
    'Occupation': 'Scientist',
    'Sleep Duration': 7,
    'Quality of Sleep': 'Good',
    'Physical Activity Level': 'Medium',
    'Stress Level': 'Medium',
    'BMI Category': 'Normal',
    'Blood Pressure Category': 'Normal',
    'Systolic': 120,
    'Diastolic': 80,
    'Heart Rate': 72,
    'Daily Steps': 8000,
    'Sleep Disorder': 'None'
}

# Test the prediction function with sample input
predict_health_status(sample_input)


ValueError: y contains previously unseen labels: 'None'

In [ ]:
import numpy as np

def predict_health_status(input_data):
    """
    Predict the health status of a user based on their input data.

    Parameters:
    input_data (dict): Dictionary containing feature values similar to original dataset structure.

    Returns:
    str: Predicted health status ('Healthy' or 'Unhealthy').
    """
    # Create a DataFrame from the input data
    input_df = pd.DataFrame([input_data])

    # Encode categorical features using the previously fitted LabelEncoders
    for column, le in label_encoders.items():
        try:
            # Attempt to transform the input data; if label is unseen, set a default value
            input_df[column] = le.transform(input_df[column].astype(str))
        except ValueError:
            # Use the most common or a default label when an unseen category appears
            input_df[column] = le.transform([le.classes_[0]])  # or specify another valid default label if needed

    # Handle missing values
    input_df['Sleep Disorder'] = imputer.transform(input_df[['Sleep Disorder']])

    # Scale features using the previously fitted StandardScaler
    input_scaled = scaler.transform(input_df)

    # Predict health status using the trained SVM model
    prediction = svm_model.predict(input_scaled)

    # Decode the prediction to its original label
    predicted_status = target_encoder.inverse_transform(prediction)[0]

    return predicted_status

# Example input data for prediction
sample_input = {
    'Gender': 'Female',
    'Age': 35,
    'Occupation': 'Scientist',
    'Sleep Duration': 7,
    'Quality of Sleep': 'Good',
    'Physical Activity Level': 'Medium',
    'Stress Level': 'Medium',
    'BMI Category': 'Normal',
    'Blood Pressure Category': 'Normal',
    'Systolic': 120,
    'Diastolic': 80,
    'Heart Rate': 72,
    'Daily Steps': 8000,
    'Sleep Disorder': 'None'  # Test for unseen value handling
}

# Test the function with this sample input
print(predict_health_status(sample_input))


Healthy


In [ ]:
import numpy as np

def get_user_input():
    """
    Prompt the user to enter values for each feature required by the model.
    Returns:
    dict: A dictionary containing user inputs for each feature.
    """
    user_data = {
        'Gender': input("Enter Gender (Male/Female): "),
        'Age': int(input("Enter Age: ")),
        'Occupation': input("Enter Occupation (e.g., Scientist, Teacher, Artist): "),
        'Sleep Duration': int(input("Enter Sleep Duration (hours): ")),
        'Quality of Sleep': input("Enter Quality of Sleep (Excellent/Good/Fair/Poor): "),
        'Physical Activity Level': input("Enter Physical Activity Level (Low/Medium/High): "),
        'Stress Level': input("Enter Stress Level (Low/Medium/High): "),
        'BMI Category': input("Enter BMI Category (Underweight/Normal/Overweight/Obese): "),
        'Blood Pressure Category': input("Enter Blood Pressure Category (Normal/Prehypertension/Hypertension): "),
        'Systolic': int(input("Enter Systolic Blood Pressure: ")),
        'Diastolic': int(input("Enter Diastolic Blood Pressure: ")),
        'Heart Rate': int(input("Enter Heart Rate: ")),
        'Daily Steps': int(input("Enter Daily Steps: ")),
        'Sleep Disorder': input("Enter Sleep Disorder (None, Insomnia, Sleep Apnea): ")
    }
    return user_data

def predict_health_status(input_data):
    """
    Predict the health status of a user based on their input data.

    Parameters:
    input_data (dict): Dictionary containing feature values similar to original dataset structure.

    Returns:
    str: Predicted health status ('Healthy' or 'Unhealthy').
    """
    # Create a DataFrame from the input data
    input_df = pd.DataFrame([input_data])

    # Encode categorical features using the previously fitted LabelEncoders
    for column, le in label_encoders.items():
        try:
            # Attempt to transform the input data; if label is unseen, set a default value
            input_df[column] = le.transform(input_df[column].astype(str))
        except ValueError:
            # Use the most common or a default label when an unseen category appears
            input_df[column] = le.transform([le.classes_[0]])  # or specify another valid default label if needed

    # Handle missing values
    input_df['Sleep Disorder'] = imputer.transform(input_df[['Sleep Disorder']])

    # Scale features using the previously fitted StandardScaler
    input_scaled = scaler.transform(input_df)

    # Predict health status using the trained SVM model
    prediction = svm_model.predict(input_scaled)

    # Decode the prediction to its original label
    predicted_status = target_encoder.inverse_transform(prediction)[0]

    return predicted_status

# Gather user input
user_input_data = get_user_input()

# Predict health status based on user input
predicted_health_status = predict_health_status(user_input_data)

print(f"Predicted Health Status: {predicted_health_status}")


Enter Gender (Male/Female): Male
Enter Age: 21
Enter Occupation (e.g., Scientist, Teacher, Artist): Teacher
Enter Sleep Duration (hours): 8
Enter Quality of Sleep (Excellent/Good/Fair/Poor): Good
Enter Physical Activity Level (Low/Medium/High): Medium
Enter Stress Level (Low/Medium/High): Medium
Enter BMI Category (Underweight/Normal/Overweight/Obese): Obese
Enter Blood Pressure Category (Normal/Prehypertension/Hypertension): Noraml
Enter Systolic Blood Pressure: 120
Enter Diastolic Blood Pressure: 80
Enter Heart Rate: 89
Enter Daily Steps: 3000
Enter Sleep Disorder (None, Insomnia, Sleep Apnea): None
Predicted Health Status: Unhealthy


In [ ]:
import numpy as np

# Reload the dataset to ensure access to the 'Nutritional Needs' column for recommendations
data = pd.read_csv('/content/nutition_updated (1).csv')

def get_user_input():
    """
    Prompt the user to enter values for each feature required by the model.
    Returns:
    dict: A dictionary containing user inputs for each feature.
    """
    user_data = {
        'Gender': input("Enter Gender (Male/Female): "),
        'Age': int(input("Enter Age: ")),
        'Occupation': input("Enter Occupation (e.g., Scientist, Teacher, Artist): "),
        'Sleep Duration': int(input("Enter Sleep Duration (hours): ")),
        'Quality of Sleep': input("Enter Quality of Sleep (Excellent/Good/Fair/Poor): "),
        'Physical Activity Level': input("Enter Physical Activity Level (Low/Medium/High): "),
        'Stress Level': input("Enter Stress Level (Low/Medium/High): "),
        'BMI Category': input("Enter BMI Category (Underweight/Normal/Overweight/Obese): "),
        'Blood Pressure Category': input("Enter Blood Pressure Category (Normal/Prehypertension/Hypertension): "),
        'Systolic': int(input("Enter Systolic Blood Pressure: ")),
        'Diastolic': int(input("Enter Diastolic Blood Pressure: ")),
        'Heart Rate': int(input("Enter Heart Rate: ")),
        'Daily Steps': int(input("Enter Daily Steps: ")),
        'Sleep Disorder': input("Enter Sleep Disorder (None, Insomnia, Sleep Apnea): ")
    }
    return user_data

def predict_health_and_nutrition(input_data):
    """
    Predict the health status and recommend nutritional needs based on user input.

    Parameters:
    input_data (dict): Dictionary containing feature values similar to the dataset structure.

    Returns:
    tuple: Predicted health status and a nutritional recommendation.
    """
    # Create a DataFrame from the input data
    input_df = pd.DataFrame([input_data])

    # Encode categorical features using the previously fitted LabelEncoders
    for column, le in label_encoders.items():
        try:
            input_df[column] = le.transform(input_df[column].astype(str))
        except ValueError:
            input_df[column] = le.transform([le.classes_[0]])  # Set to a default label if unseen

    # Handle missing values
    input_df['Sleep Disorder'] = imputer.transform(input_df[['Sleep Disorder']])

    # Scale features using the previously fitted StandardScaler
    input_scaled = scaler.transform(input_df)

    # Predict health status
    health_prediction = svm_model.predict(input_scaled)
    predicted_status = target_encoder.inverse_transform(health_prediction)[0]

    # Access nutritional recommendation based on health status
    # Use mode to get the most common recommendation for the predicted health status
    recommendation = data.loc[data['Health Status'] == predicted_status, 'Nutritional Needs'].mode()[0]

    return predicted_status, recommendation

# Gather user input
user_input_data = get_user_input()

# Predict health status and get nutritional recommendation
predicted_health_status, nutritional_recommendation = predict_health_and_nutrition(user_input_data)

print(f"Predicted Health Status: {predicted_health_status}")
print(f"Nutritional Recommendation: {nutritional_recommendation}")



Enter Gender (Male/Female): Male
Enter Age: 21
Enter Occupation (e.g., Scientist, Teacher, Artist): Engineer
Enter Sleep Duration (hours): 8
Enter Quality of Sleep (Excellent/Good/Fair/Poor): Poor
Enter Physical Activity Level (Low/Medium/High): Low
Enter Stress Level (Low/Medium/High): High
Enter BMI Category (Underweight/Normal/Overweight/Obese): Obese
Enter Blood Pressure Category (Normal/Prehypertension/Hypertension): Hypertension
Enter Systolic Blood Pressure: 150
Enter Diastolic Blood Pressure: 90
Enter Heart Rate: 110
Enter Daily Steps: 1000
Enter Sleep Disorder (None, Insomnia, Sleep Apnea): None
Predicted Health Status: Unhealthy
Nutritional Recommendation: Increase magnesium, vitamin B6, and melatonin-rich foods. | High-carb and protein diet to support energy levels. | Low-calorie, high-fiber diet to support weight loss. | Low-sodium diet to manage blood pressure. | Maintain high-carb intake to support high activity level.
